In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import string
import requests

#we are using tensorflow and keras and lstm


In [ ]:
response=requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')

In [ ]:
response.text

####1 )   here  we split whole data  with new line char

#CLEANING DATA

In [ ]:
data=response.text.split('\n')
data[0]

In [ ]:
data[253]

####2))  we update the data to remove the header from data

In [ ]:
data =data[253:]

In [ ]:
len(data)

In [ ]:
data

####3)   to make data in continuous manner before this it was in line form


In [ ]:
data=" ".join(data)
data

#### 4) removing punctuation and special char and making whole data in lower

In [ ]:
def clean_text(doc):
  tokens=doc.split()
  #remove punctuation
  table=str.maketrans(' '  , ' ', string.punctuation)
  tokens=[w.translate(table) for w in tokens ]
  #remove that are not alpha numeric or special chracter
  tokens=[word for word in tokens if word.isalpha()]
  #convert into lower letter case 
  tokens=[word.lower() for word in tokens]
  return tokens


In [ ]:
tokens=clean_text(data)
print(tokens[:50])

In [ ]:
len(tokens)

In [ ]:
len(set(tokens))

In [ ]:
#first 50 is for the input and 1 is for the output
length=50+1
#list of all senteces made from tokens joining 51 words each
lines=[]
for i in range(length,len(tokens)):
  seq=tokens[i-length:i]
  line=' '.join(seq)
  lines.append(line)
  if i > 200000:
    break

print(len(lines))


In [ ]:
lines[0]

In [ ]:
tokens[0]

In [ ]:
#0 to 50 is 51 words
tokens[50]

#LSTM MODEL BUILDING

####1)  Preparing x and y

In [ ]:
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#word embedding giving number to each word converting into integer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences=tokenizer.texts_to_sequences(lines)

In [ ]:
sequences=np.array(sequences)

In [ ]:
sequences.shape

In [ ]:
x,y=sequences[:, :-1],sequences[:,-1]

In [ ]:
x[0]

In [ ]:
y[0] 

In [ ]:
#tokenizer.word_index

In [ ]:
vocab_size=len(tokenizer.word_index)+1

In [ ]:
vocab_size

In [ ]:
y=to_categorical(y,num_classes=vocab_size)
x.shape[1]

In [ ]:
seq_length=x.shape[1]

#LSTM model

In [ ]:
model=Sequential()
model.add(Embedding( vocab_size, 50, input_length = seq_length))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(vocab_size,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'] )

In [ ]:
model.fit(x,y,batch_size = 256, epochs=200)

In [ ]:
from tensorflow import keras

In [ ]:
model.save('/saved_model')

In [ ]:
model1 = keras.models.load_model('/saved_model')

In [ ]:
model1.fit(x,y,batch_size = 256,epochs= 50 )

In [ ]:
model1.save( '/saved_model')

In [ ]:
def generate_text_seq( model, tokenizer, text_seq_length, seed_text, n_words) :
    text = []

    for _ in range( n_words ):
        encoded = tokenizer.texts_to_sequences ( [seed_text] ) [ 0]
        encoded = pad_sequences( [encoded], maxlen = text_seq_length, truncating = 'pre')

        y_predict = np.argmax( model.predict_classes( encoded ))

        predicted_word = '  '
        for word, index in tokenizer.word_index.items() :
            if index == y_predict : 
                predicted_word = word
                break
            seed_text = seed_text + '  ' + predicted_word
            text.append( predicted_word)
            
    #print(text)
    return '   '.join( text ) 


In [ ]:
seed_text = lines[1210]
seed_text

In [ ]:
generate_text_seq( model1, tokenizer,  seq_length, seed_text , 10)